# Prérequis - Importation des bibliothèques

In [ ]:
#Ignorer les warnings
import warnings
warnings.filterwarnings("ignore")

import sys
import os

sys.path.append(os.path.abspath(".."))


In [ ]:
#Importation des différentes fonctions
from src.utils import importer_fichier, extraire_infos, ask_validation
from src.verification import verifier_integrite_donnees
from src.transformation import transformation, case_uniformisation
from src.exportation import output

In [ ]:
#Importation des librairies nécessaires
import pandas as pd 
import numpy as np 
import csv
import os
import decimal


In [ ]:
decimal.getcontext().rounding = decimal.ROUND_HALF_EVEN


# 2 - Importation des données

## 2.1 - Importation du fichier brut de l'enseigne

In [ ]:
fichier = importer_fichier()

standardized_df_brut = pd.read_excel(fichier)

## 2.2 - Initialisation de la variable enseigne & date

In [ ]:
#Fonction récupération des données du nom du fichier format AAMM - enseigne (AA == Année, MM == mois)
nom_fichier = os.path.basename(fichier)

date, enseigne = extraire_infos(nom_fichier)
annee = date.split('/')[0]

## 2.3 - Importation des fichiers sharepoint (A MODIFIER AVEC L'URL DE VOS FICHIERS)

In [ ]:
#Lecture du fichiers ventes pour faire l'automatisation de l'index
table_article = pd.read_excel(r"data\input\Table articles.xlsx")

table_magasin = pd.read_excel(r"data\input\Table magasins.xlsx")

# 3 - Exploration des données

In [ ]:
#Visualisation des données
standardized_df_brut.head()

In [ ]:
standardized_df_brut.columns

In [ ]:
standardized_df_brut.dtypes

# 4 - Transformation des données

In [ ]:
pd.set_option('display.max_columns', None)


## 4.1 - Nettoyage des données brutes

In [ ]:
#Bricorama en dehors de la fonction puisque nécessite l'importation d'un fichier tiers.
if enseigne != 'bricorama' :
    df_clean, standardized_df_brut = transformation(standardized_df_brut, table_article, table_magasin, enseigne, date)
else :
    deleted_columns = ['CODE table_article ', ' CODE table_article FRN ',' LIBELLE table_article ', ' SUPPRIME ', ' PCB ', ' DN ', ' PRIX ACHAT ', ' CA HT N-1 ', ' EVOL CA % ',
        ' QTE VENTE N-1 ', ' QTE COMMANDE ', ' QTE ACHATS ', ' VAL ACHATS ',' TAUX VENTE ', ' ROTATION ', ' COUVERTURE ', ' QTE STK ', ' VAL STK '] #'Unnamed: 21'
            
        # Suppression de la ligne totaux à la fin
    standardized_df_brut.dropna(subset=[" QTE VENTE "], inplace=True)
    standardized_df_brut.drop(standardized_df_brut[(standardized_df_brut[' CODE EAN '] == " TOTAUX ")].index,inplace = True)
    standardized_df_brut.drop(standardized_df_brut[(standardized_df_brut['CODE table_article '] == " TOTAUX ")].index,inplace = True)

            #Permet de bien supprimer l'avant dernière ligne vide
    standardized_df_brut = standardized_df_brut.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    standardized_df_brut.replace("", np.nan, inplace=True)
    standardized_df_brut.dropna(how='all', inplace=True)

    df_clean = standardized_df_brut.copy()

            #La transformation en str rajoute un ".O" à la fin, on le supprime.
    df_clean[' CODE EAN '] = df_clean[' CODE EAN '].astype(str)
    df_clean[' CODE EAN '] = df_clean[' CODE EAN '].str.replace('.0','')

            #Permet d'éviter les doublons lors du merge
    df_clean = df_clean.drop(columns=deleted_columns)
    df_clean = df_clean.groupby(' CODE EAN ').sum().reset_index()

    data_last_month = None
            
    if date[5:7] != '01':
        fichier_mois_dernier = importer_fichier()
    
        data_last_month = pd.read_excel(fichier_mois_dernier)

        data_last_month.dropna(subset=[" QTE VENTE "], inplace=True)
        data_last_month.drop(data_last_month[(data_last_month['CODE table_article '] == " TOTAUX ")].index,inplace = True)

        #Permet de bien supprimer l'avant dernière ligne vide
        data_last_month = data_last_month.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        data_last_month.replace("", np.nan, inplace=True)
        data_last_month.dropna(how='all', inplace=True)
    
        data_last_month[' CODE EAN ']= data_last_month[' CODE EAN '].astype(str)
        data_last_month[' CODE EAN '] = data_last_month[' CODE EAN '].str.replace('.0','')

        #Permet d'éviter les doublons lors du merge
        data_last_month = data_last_month.drop(columns=deleted_columns)
        data_last_month = data_last_month.groupby(' CODE EAN ').sum().reset_index()

    

## 4.2 - Verification : importation EAN | Code nielsen

In [ ]:
masque_ean = pd.DataFrame()

if 'EAN' in df_clean.columns:
    masque_ean = df_clean[(df_clean['EAN'].astype(str).isin(["0", "nan", "NaN"])) | (df_clean['EAN'].isnull()) ]

if not masque_ean.empty :
    raise ValueError("GENCODE absents de la standardizedDfle table_article")

In [ ]:

masque_nielsen = pd.DataFrame()

if 'code nielsen' in df_clean.columns :
    masque_nielsen = df_clean[(df_clean['code nielsen'].astype(str).isin(["0", "nan", "NaN"])) | (df_clean['code nielsen'].isnull()) ]

if not masque_nielsen.empty :
    raise ValueError("Magasins absents de la standardizedDfle table_magasin")

## 4.3 - Structuration des données dans une structure type

In [ ]:
if enseigne != "bricorama" :
    merged_df = None
    data_last_month = None

standardized_df,standardized_df_aggregated = case_uniformisation(enseigne,df_clean,date,annee,data_last_month)

In [ ]:
standardized_df['Gencode'] =  standardized_df['Gencode'].str.replace('.0','')

#Formate la colonne date en DATE et non en STRING
standardized_df['date'] = pd.to_datetime(standardized_df['date'],format='%Y/%m')
standardized_df['date'] = standardized_df['date'].dt.strftime('%Y/%m/%d')

standardized_df = standardized_df.drop(columns=['Code_article'], axis=1)

In [ ]:
standardized_df.tail()

In [ ]:
#Reset l'index et insertion d'une colonne id
if enseigne not in ['leroy','truffaut', 'leroy es', 'castorama', 'casto mois', 'vilmorin','leclerc'] :
    standardized_df_aggregated.reset_index(inplace= True)
    
standardized_df_aggregated.insert(loc=0, column='id', value=np.random.randint(low=1, high=1000, size=len(standardized_df_aggregated)))

In [ ]:
#Formatage commun à toutes les enseignes
#Transformation de la colonne df_clean en format df_clean et en format année/mois/jours

standardized_df_aggregated['Gencode'] =  standardized_df_aggregated['Gencode'].str.replace('.0','')

standardized_df_aggregated['date'] = pd.to_datetime(standardized_df_aggregated['date'],format='%Y/%m')        
standardized_df_aggregated['date'] = standardized_df_aggregated['date'].dt.strftime('%Y/%m/%d')

In [ ]:
standardized_df_aggregated.head(2)

In [ ]:
standardized_df_aggregated.tail(2)

In [ ]:
standardized_df.head(2)

## 4.3 - Vérification : Intégrité des données

In [ ]:
volume, valeur, erreur = verifier_integrite_donnees(standardized_df_aggregated, standardized_df, standardized_df_brut, enseigne, date, annee, merged_df)

if erreur:
    raise SystemExit("Arrêt du programme suite à incohérence des données.")

## 4.5 - Vérification finale

In [ ]:
standardized_df_aggregated.head()

In [ ]:
#Vérification de tail pour vérifier les magasins avec du Ecommerce
standardized_df_aggregated.tail()

In [ ]:
standardized_df.tail()

# 5 - Exportation des données en fichier CSV

## 5.1 - Exportation du fichier df_clean.csv (df_clean agrégée)

In [ ]:
# Exportation Rajoute sur un fichier existant (mode : append), sans le nom des colonnes (header)
#rajout des quote pour les str (gencode) pour éviter les effacements de formats
os.chdir(os.path.abspath(r"chemin_du_dossier"))

valide = ''

valide = ask_validation(enseigne, date, volume, valeur)
print(valide)
file = r'data\output\data.csv'
mode = "a" #a = ajouter à la suite, W pour écraser le fichier existant

if os.path.isfile(file):
    header = False
else:
    header = True

In [ ]:
if valide == "Oui" :
    
    standardized_df_aggregated.to_csv('data\output\data.csv', index = False, mode = mode, header = header, sep = ';',decimal = ",",quoting=csv.QUOTE_NONNUMERIC)
    print("Fichier aggrégé modifié")

elif valide.capitalize() == "Test" :
    
    standardized_df_aggregated.to_csv(r"data\output\verification.csv", index = False, mode = 'w', header = True, sep = ';',decimal = ",",quoting=csv.QUOTE_NONNUMERIC)
    
    print("Fichier de vérification créé")
    
else : print("Processus annulé")

## 5.2 - Exportation du fichier enseigne

In [ ]:
if valide == "Oui" :
    output(enseigne,standardized_df)

elif valide.capitalize() == "Test" :
    print("Fichier de vérification crée à la première étape")   
    
else :
       print("Processus annulé")